In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import Normalizer
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict

In [2]:
# Read the CSV file
file_path = 'heart_disease_uci.csv'

df = pd.read_csv(file_path)
# df = df.sample(frac=1, random_state=42)
df.head()

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,Male,Cleveland,typical angina,145.0,233.0,True,lv hypertrophy,150.0,False,2.3,downsloping,0.0,fixed defect,0
1,2,67,Male,Cleveland,asymptomatic,160.0,286.0,False,lv hypertrophy,108.0,True,1.5,flat,3.0,normal,2
2,3,67,Male,Cleveland,asymptomatic,120.0,229.0,False,lv hypertrophy,129.0,True,2.6,flat,2.0,reversable defect,1
3,4,37,Male,Cleveland,non-anginal,130.0,250.0,False,normal,187.0,False,3.5,downsloping,0.0,normal,0
4,5,41,Female,Cleveland,atypical angina,130.0,204.0,False,lv hypertrophy,172.0,False,1.4,upsloping,0.0,normal,0


In [3]:
# show number of missing values
df.isna().sum()

id            0
age           0
sex           0
dataset       0
cp            0
trestbps     59
chol         30
fbs          90
restecg       2
thalch       55
exang        55
oldpeak      62
slope       309
ca          611
thal        486
num           0
dtype: int64

In [4]:
# Fill missing values
for label , content in df.items():
     if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            df[label] = content.fillna(content.median())

# Categorical data into numbers and fill missing values
for label , content in df.items():
     if not pd.api.types.is_numeric_dtype(content):
            df[label] = pd.Categorical(content).codes

In [5]:
# show number of missing values after replace
df.isna().sum()

id          0
age         0
sex         0
dataset     0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalch      0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
num         0
dtype: int64

In [6]:
df

,id,age,sex,dataset,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal,num
0,1,63,1,0,3,145.0,233.0,1,0,150.0,0,2.3,0,0.0,0,0
1,2,67,1,0,0,160.0,286.0,0,0,108.0,1,1.5,1,3.0,1,2
2,3,67,1,0,0,120.0,229.0,0,0,129.0,1,2.6,1,2.0,2,1
3,4,37,1,0,2,130.0,250.0,0,1,187.0,0,3.5,0,0.0,1,0
4,5,41,0,0,1,130.0,204.0,0,0,172.0,0,1.4,2,0.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,916,54,0,3,0,127.0,333.0,1,2,154.0,0,0.0,-1,0.0,-1,1
916,917,62,1,3,3,130.0,139.0,0,2,140.0,-1,0.5,-1,0.0,-1,0
917,918,55,1,3,0,122.0,223.0,1,2,100.0,0,0.0,-1,0.0,0,2
918,919,58,1,3,0,130.0,385.0,1,0,140.0,-1,0.5,-1,0.0,-1,0


In [7]:
# column num to binary،
for i, item in enumerate(df['num']):
  if item != 0:
    df['num'][i]= 1

<ipython-input-7-81abda5455e9>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num'][i]= 1


In [8]:
# Drop unnecessary columns
df.drop(['dataset','id'],axis= 'columns', inplace = True)

In [9]:
# Shuffle data and separate data and labels
df = df.sample(frac=1, random_state=42)
X = df.drop('num', axis='columns')
y = df['num']

In [10]:
X

,age,sex,cp,trestbps,chol,fbs,restecg,thalch,exang,oldpeak,slope,ca,thal
319,36,1,1,120.0,166.0,0,1,180.0,0,0.0,-1,0.0,-1
377,45,1,1,140.0,224.0,1,1,122.0,0,0.0,-1,0.0,-1
538,48,1,0,160.0,329.0,0,1,92.0,1,1.5,1,0.0,-1
296,59,1,0,164.0,176.0,1,0,90.0,0,1.0,1,2.0,0
531,40,0,0,150.0,392.0,0,1,130.0,0,2.0,1,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
106,59,1,0,140.0,177.0,0,1,162.0,1,0.0,2,1.0,2
270,61,1,0,140.0,207.0,0,0,138.0,1,1.9,2,1.0,2
860,75,1,0,160.0,310.0,1,1,112.0,1,2.0,0,0.0,2
435,53,0,1,140.0,216.0,0,1,142.0,1,2.0,1,0.0,-1


In [11]:
# Normalizer
transformer = Normalizer().fit(X)
X_normal = transformer.transform(X)

In [12]:
X_normal

array([[ 0.1308814 ,  0.00363559,  0.00363559, ..., -0.00363559,
         0.        , -0.00363559],
       [ 0.15283595,  0.00339635,  0.00339635, ..., -0.00339635,
         0.        , -0.00339635],
       [ 0.12622131,  0.00262961,  0.        , ...,  0.00262961,
         0.        , -0.00262961],
       ...,
       [ 0.20053118,  0.00267375,  0.        , ...,  0.        ,
         0.        ,  0.0053475 ],
       [ 0.17741934,  0.        ,  0.00334753, ...,  0.00334753,
         0.        , -0.00334753],
       [ 0.15415036,  0.        ,  0.        , ...,  0.00540878,
         0.00270439,  0.00270439]])

In [13]:
# Split data to test and train
X_train, X_test, y_train, y_test = train_test_split(X_normal,y,test_size=0.2)

In [14]:
# Initialize the AdaBoost classifier
ada_boost = AdaBoostClassifier()

# Define the parameter grid for GridSearchCV
param_grid = {'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]}

# Perform GridSearchCV to find the optimal number of estimators
grid_search = GridSearchCV(ada_boost, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Get the best number of estimators
best_estimators_n = grid_search.best_params_['n_estimators']
print(f"Optimal number of estimators: {best_estimators_n}")

# Perform cross-validation with different k values
k_values = [3, 5, 7]
for k in k_values:
    print("-"*85)
    print(f"k={k}")
    ada_boost = AdaBoostClassifier(n_estimators=best_estimators_n)

    # Perform cross-validation and get predicted labels
    y_train_pred = cross_val_predict(ada_boost, X_train, y_train, cv=k)
    y_test_pred = cross_val_predict(ada_boost, X_test, y_test, cv=k)

    # Calculate and print metrics
    accuracy_train = accuracy_score(y_train, y_train_pred)

    accuracy_test = accuracy_score(y_test, y_test_pred)
    precision = precision_score(y_test, y_test_pred)
    recall = recall_score(y_test, y_test_pred)
    f1 = f1_score(y_test, y_test_pred)

    print(f'Accuracy Train set: {accuracy_train * 100:.2f}%')
    print(f'Accuracy Test set: {accuracy_test * 100:.2f}%')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1-score: {f1:.4f}')

Optimal number of estimators: 40
-------------------------------------------------------------------------------------
k=3
Accuracy Train set: 80.71%
Accuracy Test set: 75.00%
Precision: 0.7593
Recall: 0.8039
F1-score: 0.7810
-------------------------------------------------------------------------------------
k=5
Accuracy Train set: 80.30%
Accuracy Test set: 76.63%
Precision: 0.7706
Recall: 0.8235
F1-score: 0.7962
-------------------------------------------------------------------------------------
k=7
Accuracy Train set: 80.57%
Accuracy Test set: 73.37%
Precision: 0.7477
Recall: 0.7843
F1-score: 0.7656
